In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Path to ur model
file_path = ''

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(file_path)
tokenizer = AutoTokenizer.from_pretrained(file_path)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_toxicity(text, model, tokenizer, device, threshold=0.5, max_length=128):
    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    # Tokenize input
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=max_length
    )

    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Inference
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits)  # Multi-label: use sigmoid

    probs = probs.cpu().numpy()[0]
    preds = (probs >= threshold).astype(int)

    # Determine SFW/NSFW
    nsfw_flag = "NSFW" if preds.sum() > 0 else "SFW"

    # Print results
    print(f"\nText: {text}")
    print(f"Classification: {nsfw_flag}\n")

    for label, prob, pred in zip(label_cols, probs, preds):
        print(f"{label:15}: {prob:.4f} --> {'Yes' if pred else 'No'}")

    return nsfw_flag, preds, probs


if __name__ == "__main__":
    # Put ur text here
    text = ""

    nsfw_flag, preds, probs = predict_toxicity(text, model, tokenizer, device)


/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or



Text: i'm gonna kill u motherfucker
Classification: NSFW

toxic          : 0.9961 --> Yes
severe_toxic   : 0.6383 --> Yes
obscene        : 0.9787 --> Yes
threat         : 0.9109 --> Yes
insult         : 0.8014 --> Yes
identity_hate  : 0.0320 --> No
